<h2>Digital Breakout 2022: artificial intelligence</h2>
<h3>MPTI Contest: Georeferencing aerial images to the basemap </h3>

<h4>Introduction</h4>
In the modern world, a huge number of tasks are solved with the help
of satellite and aerial images. Often, the speed and
quality of interpreting these data depends on how quickly
fires, floods and other emergencies are detected. Unfortunately,
machine vision technologies are on the first stage of being implemeted to solve
such tasks, but the need for them is constantly growing.
The solution of this problem will allow to quickly connect
images with geographical coordinates,
speed up geodetic works, quickly search for the missing
people, and control deforestation. 

<h4>Conditions</h4>
For a better understanding of the context, let's clarify the following terms:
<ul>
    <li>Basemap is an extremely large image with
a geographical reference to the terrain, i.e. the coordinates of basemap' each
pixel are known or they can be calculated.
<li>Aerial image is an image captured by a satellite, UAV,
or the aircraft. It has a significantly lower resolution
compared to the basemap. The main feature is that compared to the basemap the aerial image was taken
in other time of the year, or even in a completely
different year, and at different heights.
<li>Overlap - the position of images in which the same
area of the basemap is visible on two or more aerial images. Mutual
orientation of multi-temporal images can be done by operator manually.</ul>

The purpose of the challenge is to find a location and orientation of the image relative to the basemap.

Thanks to @boangri for code ideas.

<h3>Import modules</h3>

In [1]:
import pandas as pd 
import numpy as np
import cv2

import os
import os.path
from os import listdir
import glob

import json
import torch
import torch.nn as nn
from math import sin, cos, pi, sqrt
from time import time

import zipfile
from PIL import Image

<h3>Setup</h3>

Set MIPT_ROOT to the proper directory where we've got all the files.

In [2]:
MIPT_ROOT = os.path.abspath(r"D:\python_projects\Hakaton")

#Download train dataset from here: https://lodmedia.hb.bizmrg.com/case_files/768820/train_dataset_train.zip
TRAIN_DIR = os.path.join (MIPT_ROOT, 'train_dataset_train')

TRAIN_TIFF_ORIGIN = os.path.join (TRAIN_DIR, 'original.tiff')
TRAIN_PNG = os.path.join (TRAIN_DIR, 'train\img')
TRAIN_JSON = os.path.join (TRAIN_DIR, 'train\json')

#Download test dataset from here: https://lodmedia.hb.bizmrg.com/case_files/768820/test_dataset_test.zip
TEST_PNG = os.path.join(MIPT_ROOT, 'test_dataset_test')

SUBMIT = os.path.join (MIPT_ROOT, 'submit')

<h3>Prepare the most recent basemap</h3>

Military airbase in the bottom-right part of the original.tiff and its landscape specifics really helped to find a location. With looking through the list of the USA western coast airbases in Google Earth, finally we found it.

It's Miramar Marine Corps Airstation near San Diego: https://en.wikipedia.org/wiki/Marine_Corps_Air_Station_Miramar

We choiced the most recent from available satellite basemaps (Bing and ESRI) which time and season is close to season of test and train images, with free software SAS Planet, and downloaded basemaps on 19 zoom level which corresponds to the size and GSD of the original.tiff.
Then, we uploaded original.tiff into open-source QGIS, georeferenced ESRI and Bing basemaps to the origin, clip them by original.tiff size, and convert them into the same format as original.tiff.

In [3]:
#Download prepared a recent ESRI basemap from here: https://disk.yandex.ru/d/beHmDGZjzw_50A
TRAIN_TIFF_ESRI = os.path.join (TRAIN_DIR, 'esri_cartesian_true_final.tif') 

<h3>Settings</h3>

In [4]:
# Define shapes of inputs

for imgfile in glob.glob(TRAIN_TIFF_ORIGIN):
        img_big = Image.open(imgfile)
        print ("Dimensions of TRAIN_TIFF_ORIGIN:", img_big.size[0] ,"x",  img_big.size[1])
        
for imgfile in glob.glob(TRAIN_TIFF_ORIGIN):
        img_esri = Image.open(imgfile)
        print ("Dimensions of TRAIN_TIFF_ESRI:", img_esri.size[0] ,"x",  img_esri.size[1])
        
for imgfile in glob.glob(f"{TEST_PNG}/3.png"):
        img_small = Image.open(imgfile)
        print ("Dimensions of TEST_PNG:", img_small.size[0] ,"x",  img_small.size[1])

Dimensions of TRAIN_TIFF_ORIGIN: 10496 x 10496
Dimensions of TRAIN_TIFF_ESRI: 10496 x 10496
Dimensions of TEST_PNG: 1024 x 1024


C:\Users\permyakov\Anaconda3\lib\site-packages\PIL\Image.py:2855: DecompressionBombWarning: Image size (110166016 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [5]:
n_img = len(listdir(TEST_PNG))
print("Number of testing samples:", n_img)

Number of testing samples: 400


In [6]:
# Look at the JSON files content

data_df = pd.DataFrame({'id': [], "left_top_x": [], 'left_top_y': [], "right_bottom_x": [], 'right_bottom_y': [], 'angle': []})

json_dir = f"{TRAIN_JSON}/"
json_array = []

for _, _, files in os.walk(json_dir):
  for x in files:
    if x.endswith(".json"):
      data = json.load(open(json_dir + x))
      new_row = {'id':x.split(".")[0]+".png", 'left_top_x':data["left_top"][0], 'left_top_y':data["left_top"][1], 'right_bottom_x': data["right_bottom"][0], "right_bottom_y": data["right_bottom"][1], 'angle': data["angle"]}
      data_df = data_df.append(new_row, ignore_index=True)
    
data_df.head()

,id,left_top_x,left_top_y,right_bottom_x,right_bottom_y,angle
0,1.png,8533.0,2184.0,9834.0,2819.0,341.0
1,100.png,7078.0,4789.0,7597.0,3438.0,246.0
2,1000.png,6291.0,8856.0,6090.0,10291.0,53.0
3,1003.png,5970.0,7996.0,5719.0,6569.0,215.0
4,1004.png,4953.0,8885.0,6046.0,9836.0,356.0


In [7]:
#Setup main parameters

scale_factor = 8  # resize factor (1, 2, 4, 8...)
stride = 2 # stride of convolution (1, 2, 4, 8...)
angle_step = 1  # step of angle selection (0-360)

batch_size = 100 # split a large number of images into parts 
check = False  # check if it's time to start next batch. True is time to start
part = 0 # part of checking this time

H = img_big.size[0] #size of TRAIN_TIFF_ORIGIN
h = img_small.size[0] #size of TEST_PNGs

H8 = W8 = H // scale_factor # shape of resized TRAIN_TIFF_ORIGIN and TRAIN_TIFF_ESRI
h8 = h // scale_factor # shape of resized TEST_png

train = False 
if train:
    img_dir = TRAIN_PNG
else:
    img_dir = TEST_PNG

print("Size of TRAIN_TIFF_ORIGIN after rescaling (H8):", H8)
print("Size of TRAIN_TIFF_ESRI after rescaling (H8):", H8)
print("Size of TEST_PNGs after rescaling (h8):", h8)

Size of TRAIN_TIFF_ORIGIN after rescaling (H8): 1312
Size of TRAIN_TIFF_ESRI after rescaling (H8): 1312
Size of TEST_PNGs after rescaling (h8): 128


In [8]:
# Function to zip final jsons for submission
def zipdir(path, ziph):
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file), file)

In [9]:
# Function to get a pad (basemap)
def get_pad():
    image = cv2.imread(TRAIN_TIFF_ESRI) 
    pad0 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # convert RGB to grey to save processing time
    pad0 = cv2.resize(pad0, (H8, H8)) # resize image to 1312 x 1312
    
    #normalise image
    m, s = pad0.mean().astype(np.float32), pad0.std().astype(np.float32)  
    pad = (pad0 - m) / s
    
    #setup zero values for the most chageable area of the basemap - airbase (TOP_LEFT: 9200, -6600; BOTTOM_RIGHT: 10496, -7800) as the most typical
    for Y in range(6600 // scale_factor, 7800 // scale_factor): 
        for X in range(9200 // scale_factor, H8):
            pad[Y, X] = 0.
    return pad

# Function to get and process image samples
def get_sample(id):
    image = cv2.imread(f"{img_dir}/{id}.png")
    sample = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sample = cv2.resize(sample, (h8, h8))
    m, s = sample.mean().astype(np.float32), sample.std().astype(np.float32)
    return (sample - m) / s

# Function to get a pad (basemap) rotated with a specified angle
def rotate_pad(angle: int, pad=None):
    a = pi*angle/180
    si_, co_ = sin(a), cos(a)
    X0, Y0, L = getX0Y0(a, si_, co_)
    if pad is None:
        pad = get_pad()
    rot_pad = np.zeros((L, L), dtype=np.float32)
    for Y in range(L):
        Y1 = Y - Y0
        Ys = Y1 * si_
        Yc = Y1 * co_
        for X in range(L):
            X1 = X - X0
            x = int(X1 * co_ - Ys)
            if x < 0 or x >= H8:
                continue
            y = int(X1 * si_ + Yc)
            if y < 0 or y >= H8:
                continue
            rot_pad[Y, X] = pad[y, x]
    return rot_pad

# Function to get coordinates of image sample center in the basemap, rotated with angle "a" 
def getX0Y0(a, si_, co_):
    if a < pi/2:
        X0 = 0
        Y0 = si_
        L = sqrt(2) * sin(a + pi/4)
    elif a < pi:
        X0 = -co_
        Y0 = sqrt(2) * sin(a - pi/4)
        L = Y0
    elif a < 1.5 * pi:
        X0 = sqrt(2) * sin(a - 3*pi/4)
        Y0 = -co_
        L = X0
    else:
        X0 = -si_
        Y0 = 0
        L = sqrt(2) * cos(a + pi/4)
    return [int(H8 * X0), int(H8 * Y0), int(H8 * L)]

# Function to recalculate coordinates of point in image sample into coordinates of point in the basemap, rotated with "angle"
def get_XY(x, y, angle):
    a = pi*angle/180
    si_, co_ = sin(a), cos(a)
    X0, Y0, _ = getX0Y0(a, si_, co_)
    X = X0 + int(x * co_ + y * si_)
    Y = Y0 + int(y * co_ - x * si_)
    return [X, Y]

# Function to recalculate coordinates of point in the basemap, rotated with "angle", into coordinates of point in image sample
def get_xy(X, Y, angle):
    a = pi*angle/180
    si_, co_ = sin(a), cos(a)
    X0, Y0, _ = getX0Y0(a, si_, co_)
    x = int((X - X0) * co_ - (Y - Y0) * si_)
    y = int((Y - Y0) * co_ + (X - X0) * si_)
    return [scale_factor * x, scale_factor * y]

In [10]:
# Define neural network model
class MyModel(nn.Module):
    def __init__(self, n=400, stride=20):
        super(MyModel, self).__init__()
        self.conv = nn.Conv2d(1, n, (h8, h8), (stride, stride))  #h8 = 128 (kernel, filter size), stride = 8

    def forward(self, x):
        x = self.conv(x)
        return x

In [11]:
# Start timer
start_time = time()

# Display available device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# Define ids of testing samples
filenames = sorted(os.listdir(TEST_PNG))
all_ids = [x.split('.')[0] for x in filenames if x.endswith('png')]

# Check if it's time to start next batch (from 0 to 400 every 100 images)
for idx in range(0, n_img, batch_size): 
    if check:
        if idx != part * batch_size:
            print(f"Skipping batch {idx}")
            continue
    print(f"Start pass: {idx} - {idx+batch_size} >>>")
    ids = all_ids[idx:idx+batch_size]
    n = len(ids)
    
# Display status of modelling
    print(f"Creating model: n={n}, stride={stride}")
    model = MyModel(n=n, stride=stride)

# Display status of initializing model weights, start initializing
    print(f"Initializing model weights")
    
# Initiate model weights by image samples    
    for i, id in enumerate(ids):
        snap = get_sample(id)
        model.conv.weight.data[i, 0] = torch.from_numpy(snap)
    model.eval()
    model = model.to(device)

# Basic cycle to create json files in the "submit" folder
    scores = {}
    results = {}
    for i, id in enumerate(ids):
        scores[id] = 0.
        results[id] = {}
        
# Start calculating Pad_0, display calculation time
    since = time()
    pad0 = get_pad()
    print("Building pad_0 took %.1f sec" % (time() - since))
    
# Go over all the basemap orientations inside the cycle

# Start calculating Angle values, display calculation time   
    for angle in range(0, 360, angle_step):
        print("Angle=%d --------------" % angle)        
        since = time() # Start calculating Pad, display calculation time 
        pad = rotate_pad(angle, pad0)
        print("Building pad took %.1f sec" % (time() - since))
        
# Start calculating Inference, display calculation time       
        pad = torch.from_numpy(pad).unsqueeze_(0) #adding extra dimension
        padc = pad.to(device)
        since = time()
        with torch.no_grad(): # disable gradient calcs to reduce memory consumption
            out = model(padc)
        print("Inference took %.3f sec" % (time() - since))
        
# Start calculating shapes of pads, display shapes    
        since = time()
        Ny, Nx = out.shape[1:]
        print(f"Ny={Ny}, Nx={Nx}")
        
# Find out a maximum correspondence of position of every image sample regarding the basemap with given orientation        
        for z, id in enumerate(ids):
            dat1 = out[z]
            maxind = torch.argmax(dat1).item() # select max value
            i = maxind // Ny # floor division (x.0)
            j = maxind % Ny # modulo (0.x)
            
            val = dat1[i, j] / h8 / h8  # (h8 = 128), 
            if val > scores[id]:
                scores[id] = val
                bestX, bestY = j*stride, i*stride #search for origin coordinates 
                results[id]['left_top'] = get_xy(bestX, bestY, angle)
                results[id]['right_top'] = get_xy(bestX + h8, bestY, angle)
                results[id]['left_bottom'] = get_xy(bestX, bestY + h8, angle)
                results[id]['right_bottom'] = get_xy(bestX + h8, bestY + h8, angle)
                results[id]['angle'] = angle
        print("Search took %.3f sec" % (time() - since))
    print("------------------------------")
    
# Create json files for all the images in the cycle    
    for id in ids:
        print("id=%s Score: %.3f" % (id, scores[id]), results[id])
        filename = f"submit/{id}.json"
        with open(filename, "w") as f:
            json.dump(results[id], f)
            
# Zip json files to prepare them for submission checking
zipname = f'perm-{scale_factor}-{stride}-{angle_step}-esri.zip'
with zipfile.ZipFile(zipname, 'w', zipfile.ZIP_DEFLATED) as zf:
    zipdir('submit/', zf)
print("Created ", zipname)
print("Completed. It took %.1f secs." % (time() - start_time))

Using device: cpu
Start pass: 0 - 100 >>>
Creating model: n=100, stride=2
Initializing model weights
Building pad_0 took 0.6 sec
Angle=0 --------------
Building pad took 1.2 sec
Inference took 7.167 sec
Ny=593, Nx=593
Search took 0.076 sec
Angle=1 --------------
Building pad took 1.3 sec
Inference took 6.881 sec
Ny=604, Nx=604
Search took 0.089 sec
Angle=2 --------------
Building pad took 1.3 sec
Inference took 9.200 sec
Ny=615, Nx=615
Search took 0.102 sec
Angle=3 --------------
Building pad took 1.3 sec
Inference took 7.765 sec
Ny=626, Nx=626
Search took 0.093 sec
Angle=4 --------------
Building pad took 1.3 sec
Inference took 8.604 sec
Ny=637, Nx=637
Search took 0.095 sec
Angle=5 --------------
Building pad took 1.3 sec
Inference took 8.045 sec
Ny=647, Nx=647
Search took 0.101 sec
Angle=6 --------------
Building pad took 1.3 sec
Inference took 8.688 sec
Ny=657, Nx=657
Search took 0.102 sec
Angle=7 --------------
Building pad took 1.3 sec
Inference took 11.116 sec
Ny=668, Nx=668
Sear

Building pad took 1.5 sec
Inference took 12.557 sec
Ny=763, Nx=763
Search took 0.141 sec
Angle=73 --------------
Building pad took 1.6 sec
Inference took 11.876 sec
Ny=756, Nx=756
Search took 0.135 sec
Angle=74 --------------
Building pad took 1.5 sec
Inference took 12.273 sec
Ny=748, Nx=748
Search took 0.131 sec
Angle=75 --------------
Building pad took 1.5 sec
Inference took 11.651 sec
Ny=740, Nx=740
Search took 0.127 sec
Angle=76 --------------
Building pad took 1.5 sec
Inference took 11.214 sec
Ny=732, Nx=732
Search took 0.129 sec
Angle=77 --------------
Building pad took 1.4 sec
Inference took 11.066 sec
Ny=723, Nx=723
Search took 0.127 sec
Angle=78 --------------
Building pad took 1.4 sec
Inference took 11.341 sec
Ny=715, Nx=715
Search took 0.123 sec
Angle=79 --------------
Building pad took 1.4 sec
Inference took 10.064 sec
Ny=706, Nx=706
Search took 0.125 sec
Angle=80 --------------
Building pad took 1.4 sec
Inference took 10.826 sec
Ny=696, Nx=696
Search took 0.122 sec
Angle=8

Building pad took 1.8 sec
Inference took 15.200 sec
Ny=850, Nx=850
Search took 0.171 sec
Angle=146 --------------
Building pad took 1.8 sec
Inference took 14.618 sec
Ny=847, Nx=847
Search took 0.163 sec
Angle=147 --------------
Building pad took 1.7 sec
Inference took 14.134 sec
Ny=844, Nx=844
Search took 0.162 sec
Angle=148 --------------
Building pad took 1.7 sec
Inference took 14.332 sec
Ny=840, Nx=840
Search took 0.176 sec
Angle=149 --------------
Building pad took 1.7 sec
Inference took 14.107 sec
Ny=837, Nx=837
Search took 0.169 sec
Angle=150 --------------
Building pad took 1.8 sec
Inference took 15.829 sec
Ny=833, Nx=833
Search took 0.170 sec
Angle=151 --------------
Building pad took 1.8 sec
Inference took 13.854 sec
Ny=828, Nx=828
Search took 0.159 sec
Angle=152 --------------
Building pad took 1.7 sec
Inference took 13.232 sec
Ny=824, Nx=824
Search took 0.160 sec
Angle=153 --------------
Building pad took 1.7 sec
Inference took 12.975 sec
Ny=819, Nx=819
Search took 0.163 sec

Inference took 15.698 sec
Ny=855, Nx=855
Search took 0.168 sec
Angle=218 --------------
Building pad took 1.8 sec
Inference took 15.710 sec
Ny=857, Nx=857
Search took 0.172 sec
Angle=219 --------------
Building pad took 1.8 sec
Inference took 16.273 sec
Ny=859, Nx=859
Search took 0.170 sec
Angle=220 --------------
Building pad took 1.9 sec
Inference took 15.385 sec
Ny=861, Nx=861
Search took 0.176 sec
Angle=221 --------------
Building pad took 1.8 sec
Inference took 15.354 sec
Ny=862, Nx=862
Search took 0.170 sec
Angle=222 --------------
Building pad took 1.8 sec
Inference took 14.546 sec
Ny=863, Nx=863
Search took 0.177 sec
Angle=223 --------------
Building pad took 1.8 sec
Inference took 15.584 sec
Ny=864, Nx=864
Search took 0.177 sec
Angle=224 --------------
Building pad took 1.8 sec
Inference took 15.880 sec
Ny=864, Nx=864
Search took 0.170 sec
Angle=225 --------------
Building pad took 1.8 sec
Inference took 17.192 sec
Ny=864, Nx=864
Search took 0.171 sec
Angle=226 --------------


Building pad took 1.6 sec
Inference took 11.768 sec
Ny=777, Nx=777
Search took 0.140 sec
Angle=291 --------------
Building pad took 1.7 sec
Inference took 13.555 sec
Ny=784, Nx=784
Search took 0.139 sec
Angle=292 --------------
Building pad took 1.6 sec
Inference took 13.405 sec
Ny=790, Nx=790
Search took 0.145 sec
Angle=293 --------------
Building pad took 1.6 sec
Inference took 12.559 sec
Ny=797, Nx=797
Search took 0.154 sec
Angle=294 --------------
Building pad took 1.6 sec
Inference took 13.106 sec
Ny=803, Nx=803
Search took 0.148 sec
Angle=295 --------------
Building pad took 1.7 sec
Inference took 14.623 sec
Ny=808, Nx=808
Search took 0.151 sec
Angle=296 --------------
Building pad took 1.7 sec
Inference took 12.943 sec
Ny=814, Nx=814
Search took 0.160 sec
Angle=297 --------------
Building pad took 1.7 sec
Inference took 15.497 sec
Ny=819, Nx=819
Search took 0.152 sec
Angle=298 --------------
Building pad took 1.7 sec
Inference took 14.380 sec
Ny=824, Nx=824
Search took 0.159 sec

KeyboardInterrupt: 